# Graph for STD and terminal peak
to illustrate the different schedules we will plot:
- 1hr grouped STD dep pax
- 1hr rolling average terminal show-up

## imports

In [ ]:
# tutorial.ipynb
# import some useful librairies
import os  # to use some OS commands
from pathlib import Path  # to handle Paths

import matplotlib.dates as mdates  # to handle dates in plots
import matplotlib.pyplot as plt  # for plot
import matplotlib.ticker as mtick  # to customize tickers
import numpy as np  # for math
import pandas as pd  # "excel but better"
from decouple import AutoConfig  # to get info from .env file
from scipy.interpolate import interp1d  # to interpolate 1d function

from src.utils.profiles import generate_dep_Pax_Counters
from src.utils.profiles_from_schedule import generate_dep_Pax_Counters as custom_generate_dep_Pax_Counters

In [ ]:
# to auto-reload the imports
# if we change something in our functions
%load_ext autoreload
%autoreload 2

%load_ext nb_black

## Calculations

In [ ]:
# calculate df_Pax
df_Pax_2025, df_Counters_2025 = generate_dep_Pax_Counters(
    target_peak=3880,  # peak value for FY2025 schedule forecast from aero
    terminal="T1",
)

# retrieve processed 2059 schedule
schedule_2059_path = (
    Path(os.getcwd())
    / "../../../data/processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
)

df_Pax_2059, df_Counters_2059 = custom_generate_dep_Pax_Counters(
    path_to_schedule=schedule_2059_path,
    terminal="T1",
    date_str="2017-03-19",
)

In [ ]:
# reorganize data for plotting
dct_df_Pax = {
    "2025": df_Pax_2025,
    "2059": df_Pax_2059,
}

for year in dct_df_Pax.keys():
    dct_df_Pax[year]["N_Pax"] = 1

dct_plot = {
    "STD_2025": np.nan,
    "STD_2059": np.nan,
    "terminal_2025": np.nan,
    "terminal_2059": np.nan,
}

for year in dct_df_Pax.keys():

    dct_plot["STD_{}".format(year)] = (
        dct_df_Pax[year]
        .set_index("Scheduled Time", drop=False)
        .sort_index()
        .resample("60min")
        .agg("sum")
    )

    dct_plot["terminal_{}".format(year)] = (
        dct_df_Pax[year]
        .set_index("time", drop=False)
        .sort_index()
        .resample("5min")
        .agg("sum")
        .rolling(window=12, center=True)
        .sum()
    )

In [ ]:
# plot

# plot param
xmin = pd.to_datetime("2020-10-13 00:00:00")
xmax = pd.to_datetime("2020-10-14 00:00:00")
plt.rcParams.update({"figure.autolayout": True})
hours = mdates.HourLocator(interval=1)
half_hours = mdates.MinuteLocator(byminute=[0, 30], interval=1)
h_fmt = mdates.DateFormatter("%H:%M")

for year in dct_df_Pax.keys():

    fig, ax = plt.subplots(figsize=(7, 4))
    ax2 = ax.twiny()

    ax.plot(dct_plot["terminal_{}".format(year)], label="terminal show-up")
    ax2.bar(
        x=dct_plot["STD_{}".format(year)].index,
        height=dct_plot["STD_{}".format(year)]["N_Pax"],
        width=1 / 24,
        color="green",
        alpha=0.6,
        label="schedule STD",
    )

    ax.legend(frameon=False, loc="upper left")
    ax2.legend(frameon=False, loc="upper right")

    ax2.axes.get_xaxis().set_visible(False)

    # formatting
    ax.set_xlim((xmin, xmax))
    ax.set_xticks(dct_plot["STD_{}".format(year)].index.to_list())
    ax.set_xticklabels(ax.get_xticks(), rotation=45, **{"horizontalalignment": "right"})
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(h_fmt)
    ax.xaxis.set_minor_locator(half_hours)

    # scale y axis to leave more space
    ax.set_ylim(bottom=0, top=8500)

## Bonus: why is STD "schedule peak" not the same as the input of generate_pax function?
answer: the true definition of STD peak is not the sum over a full hour (eg.0900-1000) but the max of the rolling average over one hour. (eg. it could be 0842-0942) \
See calculation below

In [ ]:
(
    dct_df_Pax["2025"]
    .set_index("Scheduled Time", drop=False)
    .sort_index()
    .resample("1min")
    .agg("sum")
    .rolling(60)
    .sum()
    .max()
)